In [ ]:
import pandas as pd
import time,datetime
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
order = pd.read_csv('order.csv',dtype={'order_id':str,'group_id':str})
group = pd.read_csv('group.csv',dtype={'group_id':str})
# day_schedule = pd.read_csv('day_schedule.csv',dtype={'group_id':str})
# airline = pd.read_csv('airline.csv',dtype={'group_id':str})

train = pd.read_csv('training-set.csv',dtype={'order_id':str})
test = pd.read_csv('testing-set.csv',dtype={'order_id':str})

In [ ]:
month = {'Jan': '01', 'Feb': '02' , 'Mar': '03' , 'Apr': '04',
         'May': '05', 'Jun': '06' , 'Jul': '07' , 'Aug': '08',
         'Sep': '09', 'Oct': '10' , 'Nov': '11' , 'Dec': '12' }
def Convert_Date(x):
    
    Year='20'+x[-2:]
    Month=month[x[-6:-3]]
    Day=x[:-7]

    return pd.to_datetime(Year+'-'+Month+'-'+Day)

In [ ]:
promotion_dict = dict(zip(group.promotion_prog.unique(),range(len(group.promotion_prog.unique()))))
group['promotion_prog'] = group['promotion_prog'].apply(lambda x: promotion_dict[x])

product_name_dict = dict(zip(group.product_name.unique(),range(len(group.product_name.unique()))))
group['product_name'] = group['promotion_prog'].apply(lambda x: product_name_dict[x])

group['begin_date']=group.begin_date.apply(lambda x: Convert_Date(x))
group['sub_line']= group.sub_line.apply(lambda x: int(x[14:]))
group['area']= group.area.apply(lambda x: int(x[11:]))

order = order.merge(group, on='group_id')

In [ ]:
ts = time.time()

group_dict = dict(zip(order.group_id.unique(),range(len(order.group_id.unique()))))
order['group_id'] = order['group_id'].apply(lambda x: group_dict[x])

order['order_date']=order.order_date.apply(lambda x: Convert_Date(x))
order['source_1']=order.source_1.apply(lambda x: int(x[11:]))
order['source_2']=order.source_2.apply(lambda x: int(x[11:]))
order['unit']=order.unit.apply(lambda x: int(x[11:]))
order['return_date'] = order.begin_date + order.days.apply(lambda x:datetime.timedelta(days=x))

order['order_date_dayofweek']=order.order_date.dt.dayofweek
order['order_date_dayofyear'] = order.order_date.dt.dayofyear
order['order_date_dayofmonth'] = order.order_date.dt.day
order['order_date_weekofyear'] = order.order_date.dt.week
order['order_date_monthdays'] = order.order_date.dt.daysinmonth
order['order_date_month'] = order.order_date.dt.month
order['order_date_quarter'] = order.order_date.dt.quarter

order['begin_date_dayofweek']=order.begin_date.dt.dayofweek
order['begin_date_dayofyear'] = order.begin_date.dt.dayofyear
order['begin_date_dayofmonth'] = order.begin_date.dt.day
order['begin_date_weekofyear'] = order.begin_date.dt.week
order['begin_date_monthdays'] = order.begin_date.dt.daysinmonth
order['begin_date_month'] = order.begin_date.dt.month
order['begin_date_quarter'] = order.begin_date.dt.quarter

order['return_date_dayofweek']=order.return_date.dt.dayofweek
order['return_date_dayofyear'] = order.return_date.dt.dayofyear
order['return_date_dayofmonth'] = order.return_date.dt.day
order['return_date_weekofyear'] = order.return_date.dt.week
order['return_date_monthdays'] = order.return_date.dt.daysinmonth
order['return_date_month'] = order.return_date.dt.month
order['return_date_quarter'] = order.return_date.dt.quarter

order['begin_date_from2016'] = (order.begin_date - datetime.datetime(2016,1,1)).dt.days.astype(float)
order['order_date_from2016'] = (order.order_date - datetime.datetime(2016,1,1)).dt.days.astype(float)

order['predays'] = (order['begin_date']-order['order_date']).dt.days
order['preweeks'] = order.predays/7
order['premonths'] = order.preweeks/4
# order['preyears'] = order.premonths/12

order['ticketsischeap_days'] = order.predays.apply(lambda x:1 if ((53-14)<x)&(x<53+14) else 0)
order['ticketsischeap_week'] = order.order_date_dayofweek.apply(lambda x:1 if x==1 else 0)
order['ticketsisexpen_week'] = order.order_date_dayofweek.apply(lambda x:1 if x==4 else 0)
order['order_holiday'] = order.order_date_dayofweek.apply(lambda x:1 if x==4 or x==5 or x==6 else 0)

time.time()-ts

In [ ]:
ts = time.time()
columns = ['group_id','sub_line', 'area', 'days','unit','people_amount','source_1','source_2','promotion_prog','product_name',
          'order_date_month','order_date_dayofyear','order_date_weekofyear','order_date_dayofmonth','order_date_dayofweek',
          'begin_date_dayofweek','begin_date_dayofyear','begin_date_dayofmonth','begin_date_weekofyear','begin_date_month',
          'return_date_dayofweek','return_date_dayofyear','return_date_dayofmonth','return_date_weekofyear','return_date_month',
          'begin_date_from2016','order_date_from2016','ticketsischeap_days','ticketsischeap_week','ticketsisexpen_week','order_holiday',
          'begin_date_quarter']
methods = ['median','mean','max','min','skew']
for column in columns:
    print(column)
    for method in methods:  
        order[column+'_price_'+method] = order.groupby(column)['price'].transform(method).astype(float)
        order[column+'_predays_'+method] = order.groupby(column)['predays'].transform(method).astype(float)
    order[column+'_price_diff_median'] = order['price'] - order[column+'_price_median']
    order[column+'_price_diff_mean'] = order['price'] - order[column+'_price_mean']
    order[column+'_predays_diff_median'] = order['predays'] - order[column+'_predays_median']
    order[column+'_predays_diff_mean'] = order['predays'] - order[column+'_predays_mean']
time.time()-ts

In [ ]:
# round 1 feat importance == 0 in 4 folds
del_column=['order_holiday_price_mean',
 'order_holiday_price_skew',
 'order_holiday_predays_min',
 'order_holiday_price_min',
 'order_holiday_predays_max',
 'order_holiday_price_max',
 'order_holiday_predays_mean',
 'ticketsischeap_days_price_median',
 'group_id_price_min',
 'order_holiday_predays_median',
 'order_date_quarter',
 'ticketsisexpen_week_predays_diff_median',
 'group_id_price_diff_median',
 'group_id_price_diff_mean',
 'order_holiday_predays_skew',
 'ticketsisexpen_week_price_skew',
 'return_date_quarter',
 'begin_date_quarter_predays_skew',
 'begin_date_quarter_price_skew',
 'begin_date_quarter_predays_min',
 'begin_date_quarter_predays_max',
 'begin_date_quarter',
 'begin_date_quarter_price_median',
 'source_2_price_skew',
 'ticketsisexpen_week_predays_skew',
 'ticketsisexpen_week_predays_min',
 'ticketsischeap_days_predays_median',
 'ticketsischeap_days_predays_skew',
 'ticketsischeap_week_predays_median',
 'source_1_predays_min',
 'source_1_predays_skew',
 'source_1_price_skew',
 'ticketsischeap_days_price_skew',
 'ticketsisexpen_week_price_min',
 'ticketsischeap_days_predays_min',
 'ticketsischeap_days_price_min',
 'ticketsischeap_days_predays_max',
 'ticketsischeap_days_price_max',
 'ticketsischeap_days_predays_mean',
 'ticketsischeap_days_price_mean',
 'ticketsischeap_week_price_mean',
 'ticketsischeap_week_price_max',
 'ticketsischeap_week_predays_max',
 'ticketsischeap_week_price_min',
 'ticketsischeap_week_predays_min',
 'ticketsischeap_week_predays_skew',
 'source_1_price_max',
 'ticketsischeap_week_predays_diff_median',
 'ticketsisexpen_week_predays_median',
 'ticketsisexpen_week_predays_mean',
 'ticketsisexpen_week_price_max',
 'ticketsisexpen_week_predays_max',
 'ticketsischeap_week_price_skew']

In [ ]:
order.drop(columns=del_column,inplace=True)

In [ ]:
# round 1 每一個fold前70個之聯集相關feat 15/16
price_column=['order_date_dayofyear', 'unit', 'area', 'product_name', 'return_date_dayofyear', 'sub_line', 'group_id',
              'people_amount', 'order_date_dayofmonth', 'promotion_prog', 'source_1', 'source_2', 'begin_date_dayofyear',
              'order_date_from2016', 'begin_date_from2016']

predays_column=['order_date_dayofyear', 'unit', 'area', 'return_date_dayofweek', 'product_name', 'return_date_dayofyear',
                'group_id', 'people_amount', 'order_date_dayofmonth', 'promotion_prog', 'order_date_dayofweek', 'source_1',
                'source_2', 'begin_date_dayofyear', 'order_date_from2016', 'begin_date_from2016']

In [ ]:
ts = time.time()
methods = ['median','mean']
for i,column in enumerate(price_column):
    print(i,column)
    for j in range(i+1,len(price_column)):
        print(j)
        for method in methods: 
            order[column+'-'+price_column[j]+'-price_'+method] = order.groupby([column,price_column[j]])['price'].transform(method).astype(float)
            order[column+'-'+price_column[j]+'-price_diff_'+method] = order['price'] - order[column+'-'+price_column[j]+'-price_'+method]
for i,column in enumerate(predays_column):
    print(i,column)
    for j in range(i+1,len(predays_column)): 
        print(j)
        for method in methods:  
            order[column+'-'+predays_column[j]+'-predays_'+method] = order.groupby([column,predays_column[j]])['predays'].transform(method).astype(float)
            order[column+'-'+predays_column[j]+'-predays_diff_'+method] = order['predays'] - order[column+'-'+predays_column[j]+'-predays_'+method]
time.time()-ts

In [ ]:
order = pd.read_pickle('order1340_new.pkl')#round 2 -> 3

In [ ]:
del_column = ['begin_date_dayofweek_price_diff_mean',
 'people_amount_predays_diff_median',
 'begin_date_month_predays_max',
 'product_name-begin_date_dayofyear-price_mean',
 'sub_line-group_id-price_diff_mean',
 'return_date_dayofyear-group_id-predays_diff_mean',
 'begin_date_dayofyear-begin_date_from2016-predays_diff_median',
 'order_date_dayofmonth-order_date_from2016-price_diff_median',
 'source_1-source_2-predays_diff_mean',
 'order_date_from2016-begin_date_from2016-predays_diff_mean',
 'return_date_dayofweek_predays_diff_mean',
 'order_holiday_predays_diff_mean',
 'group_id-order_date_from2016-price_median',
 'product_name-group_id-predays_diff_median',
 'order_date_dayofmonth-source_1-price_diff_mean',
 'product_name_price_mean',
 'group_id-promotion_prog-predays_diff_median',
 'return_date_month',
 'return_date_month_predays_min',
 'begin_date_quarter_predays_mean',
 'order_date_from2016-begin_date_from2016-predays_median',
#  'price',
 'return_date_dayofweek_predays_min',
 'return_date_dayofmonth_price_diff_mean',
 'people_amount-source_2-predays_diff_mean',
 'sub_line-group_id-price_diff_median',
 'order_date_dayofyear-return_date_dayofyear-predays_mean',
 'sub_line-source_1-price_mean',
 'ticketsischeap_days_price_diff_mean',
 'product_name-source_2-price_mean',
 'return_date_dayofyear-order_date_from2016-predays_median',
 'return_date_dayofyear-promotion_prog-price_diff_median',
 'product_name-promotion_prog-price_mean',
 'order_date_dayofmonth_predays_min',
 'source_1-order_date_from2016-price_diff_median',
 'promotion_prog-begin_date_from2016-price_diff_median',
 'order_date_dayofweek_price_diff_mean',
 'begin_date_dayofmonth_predays_diff_median',
 'source_2_predays_diff_mean',
 'product_name-group_id-predays_mean',
 'area-group_id-price_diff_mean',
 'group_id-source_1-price_diff_mean',
 'days_price_max',
 'order_date_dayofweek_predays_diff_median',
 'begin_date_quarter_price_max',
 'product_name-group_id-price_diff_median',
 'area-group_id-price_mean',
 'order_date_dayofweek_price_diff_median',
 'order_holiday_price_median',
 'order_date_month_predays_min',
 'group_id-source_2-price_median',
 'area-sub_line-price_median',
 'ticketsischeap_week_price_diff_median',
 'order_date_from2016_price_diff_median',
 'order_date_month_predays_median',
 'begin_date_dayofweek_price_mean',
 'return_date_dayofweek-order_date_dayofmonth-predays_diff_median',
 'order_date_from2016-begin_date_from2016-predays_mean',
 'order_date_dayofmonth-order_date_from2016-predays_diff_median',
 'source_2_price_diff_median',
 'product_name-order_date_dayofmonth-price_mean',
 'order_date_weekofyear_price_diff_median',
 'order_date_month_price_skew',
 'begin_date_quarter_price_mean',
 'order_date_dayofmonth-begin_date_dayofyear-predays_diff_median',
 'product_name-begin_date_from2016-price_mean',
 'promotion_prog-order_date_from2016-price_median',
 'sub_line-promotion_prog-price_mean',
 'group_id-order_date_dayofmonth-price_mean',
 'return_date_month_price_median',
 'group_id-order_date_from2016-predays_diff_median',
 'product_name-begin_date_dayofyear-predays_median',
 'order_date_dayofyear-begin_date_from2016-predays_diff_mean',
 'product_name-return_date_dayofyear-price_median',
 'days_predays_min',
 'order_date_dayofyear_price_diff_mean',
 'return_date_dayofyear-group_id-predays_median',
 'product_name-begin_date_dayofyear-price_diff_mean',
 'product_name-begin_date_dayofyear-price_median',
 'order_date_dayofweek-order_date_from2016-predays_median',
 'source_2_predays_diff_median',
 'return_date_dayofweek-source_1-predays_diff_median',
 'group_id-begin_date_from2016-predays_diff_median',
 'promotion_prog-begin_date_dayofyear-price_diff_median',
 'order_date_dayofyear-source_2-price_diff_median',
 'return_date_dayofyear-group_id-price_diff_mean',
 'product_name-begin_date_dayofyear-price_diff_median',
 'begin_date_quarter_price_min',
 'order_holiday_price_diff_mean',
 'days_predays_mean',
 'ticketsischeap_week_price_median',
 'area-promotion_prog-price_median',
 'people_amount_predays_diff_mean',
 'source_2_predays_max',
 'group_id-source_1-price_diff_median',
 'order_date_dayofyear-order_date_dayofweek-predays_diff_mean',
 'product_name-group_id-price_diff_mean',
 'group_id-order_date_from2016-predays_median',
 'return_date_dayofweek_price_diff_mean',
 'group_id-promotion_prog-price_diff_mean',
 'return_date_dayofyear-group_id-price_mean',
 'unit-product_name-price_median',
 'return_date_dayofweek-source_2-predays_diff_median',
 'group_id-begin_date_dayofyear-price_median',
 'order_date_dayofyear-order_date_from2016-predays_diff_mean',
 'begin_date_monthdays',
 'return_date_dayofyear-group_id-predays_mean',
 'order_date_dayofyear-order_date_dayofweek-predays_diff_median',
 'promotion_prog_price_mean',
 'order_date_dayofmonth-source_2-predays_diff_mean',
 'order_date_dayofweek_predays_min',
 'source_2_predays_min',
 'return_date_dayofyear_predays_diff_mean',
 'area-sub_line-price_mean',
 'begin_date_month_predays_median',
 'order_date_dayofmonth-source_2-price_diff_median',
 'product_name-order_date_from2016-price_mean',
 'return_date_dayofweek_predays_max',
 'unit-group_id-price_median',
 'begin_date_quarter_price_diff_median',
 'group_id-promotion_prog-predays_median',
 'order_date_from2016-begin_date_from2016-price_median',
 'begin_date_dayofyear-begin_date_from2016-predays_diff_mean',
 'order_date_dayofyear-product_name-price_median',
 'order_date_dayofyear-group_id-predays_diff_mean',
 'return_date_monthdays',
 'begin_date_dayofmonth_price_diff_median',
 'group_id-people_amount-price_median',
 'return_date_month_predays_median',
 'order_date_dayofmonth_price_diff_median',
 'order_date_dayofweek-order_date_from2016-predays_mean',
 'product_name-return_date_dayofyear-price_diff_median',
 'source_2-order_date_from2016-predays_diff_median',
 'product_name-order_date_from2016-price_median',
 'source_1_predays_diff_median',
 'product_name-people_amount-price_mean',
 'order_date_dayofmonth_predays_diff_mean',
 'order_holiday',
 'ticketsischeap_days_predays_diff_mean',
 'begin_date_dayofweek_predays_diff_median',
 'order_date_dayofmonth-order_date_from2016-price_diff_mean',
 'sub_line-group_id-price_mean',
 'begin_date_dayofyear-begin_date_from2016-predays_median',
 'order_date_dayofmonth-order_date_from2016-predays_mean',
 'order_date_dayofyear-order_date_from2016-price_diff_mean',
 'return_date_dayofmonth_price_diff_median',
 'product_name-people_amount-price_median',
 'order_date_dayofmonth_predays_diff_median',
 'promotion_prog-begin_date_from2016-price_diff_mean',
 'area-promotion_prog-price_mean',
 'promotion_prog_price_median',
 'group_id-order_date_from2016-price_diff_median',
 'days_price_median',
 'order_date_dayofmonth_predays_median',
 'product_name-promotion_prog-price_median',
 'group_id-begin_date_from2016-price_median',
 'order_date_dayofyear-group_id-price_diff_median',
 'return_date_dayofweek_price_min',
 'source_1_price_diff_mean',
 'sub_line-source_2-price_median',
 'order_date_dayofweek_predays_diff_mean',
 'begin_date_month',
 'group_id-source_1-price_mean',
 'return_date_dayofweek_predays_median',
 'begin_date_dayofyear-order_date_from2016-predays_median',
 'sub_line_predays_median',
 'order_date_dayofmonth-order_date_from2016-predays_median',
 'ticketsisexpen_week_price_mean',
 'order_date_dayofweek-order_date_from2016-predays_diff_median',
 'order_date_monthdays',
 'return_date_dayofweek_price_median',
 'product_name-group_id-price_median',
 'people_amount-promotion_prog-price_median',
 'order_date_dayofmonth-source_1-predays_diff_mean',
 'source_1_predays_max',
 'area-group_id-price_median',
 'group_id-begin_date_from2016-price_diff_mean',
 'days_price_mean',
 'ticketsisexpen_week_predays_diff_mean',
 'group_id-source_1-price_median',
 'return_date_dayofweek-source_1-predays_diff_mean',
 'source_1_price_min',
 'ticketsisexpen_week',
 'group_id-promotion_prog-price_mean',
 'group_id-begin_date_from2016-predays_median',
 'begin_date_dayofweek_price_diff_median',
 'area-source_2-price_median',
 'group_id-begin_date_from2016-predays_mean',
 'product_name-sub_line-price_median',
 'order_date_dayofweek-source_2-predays_diff_median',
 'unit-group_id-price_mean',
 'ticketsischeap_week_price_diff_mean',
 'order_date_dayofyear-source_1-price_diff_median',
 'order_date_dayofmonth-order_date_from2016-price_mean',
 'order_holiday_predays_diff_median',
 'group_id-promotion_prog-predays_mean',
 'return_date_dayofweek-group_id-predays_diff_mean',
 'preweeks',
 'unit-group_id-price_diff_median',
 'source_1-source_2-predays_diff_median',
 'order_date_dayofmonth_price_median',
 'source_1-source_2-price_diff_mean',
 'order_date_dayofyear-promotion_prog-price_median',
 'source_1-source_2-price_median',
 'promotion_prog-source_2-price_mean',
 'return_date_dayofweek-group_id-predays_diff_median',
 'order_date_dayofyear-begin_date_dayofyear-predays_mean',
 'begin_date_month_price_median',
 'begin_date_dayofyear-begin_date_from2016-price_diff_median',
 'return_date_dayofmonth_predays_diff_median',
 'group_id-people_amount-price_diff_mean',
 'order_date_dayofweek_price_min',
 'order_date_dayofyear-product_name-price_mean',
 'group_id-promotion_prog-price_diff_median',
 'product_name_price_median',
 'product_name-order_date_dayofmonth-price_median',
 'order_date_dayofweek_price_median',
 'sub_line_price_mean',
 'return_date_weekofyear_price_diff_median',
 'group_id-begin_date_from2016-price_diff_median',
 'return_date_dayofweek_price_diff_median',
 'promotion_prog-begin_date_dayofyear-price_mean',
 'group_id-source_2-price_diff_mean',
 'group_id-begin_date_dayofyear-predays_diff_mean',
 'return_date_month_price_diff_median',
 'order_date_dayofmonth-source_1-price_diff_median',
 'begin_date_dayofweek_predays_min',
 'order_date_dayofmonth_price_diff_mean',
 'ticketsisexpen_week_price_median',
 'order_date_dayofyear-begin_date_dayofyear-predays_median',
 'area-group_id-predays_mean',
 'people_amount-source_1-price_diff_median',
 'group_id-begin_date_dayofyear-predays_median',
 'group_id-order_date_from2016-predays_diff_mean',
 'source_1_price_mean',
 'group_id-people_amount-price_mean',
 'group_id_price_mean',
 'begin_date_month_price_diff_median',
 'return_date_weekofyear_predays_min',
 'source_1-source_2-price_diff_median',
 'product_name-sub_line-price_diff_median',
 'group_id-promotion_prog-price_median',
 'order_date_dayofmonth-order_date_from2016-price_median',
 'order_date_month_price_median',
 'promotion_prog-begin_date_dayofyear-price_median',
 'group_id_price_max',
 'order_date_dayofyear-order_date_from2016-predays_diff_median',
 'product_name-group_id-price_mean',
 'area-product_name-price_mean',
 'order_date_dayofweek-order_date_from2016-predays_diff_mean',
 'order_date_dayofmonth-order_date_from2016-predays_diff_mean',
 'source_1_predays_diff_mean',
 'group_id-source_2-price_mean',
 'order_date_dayofyear-order_date_from2016-predays_median',
 'group_id-begin_date_dayofyear-price_mean',
 'ticketsisexpen_week_price_diff_mean',
 'people_amount-source_1-price_diff_mean',
 'promotion_prog-begin_date_from2016-price_median',
 'order_holiday_price_diff_median',
 'begin_date_dayofyear-order_date_from2016-predays_diff_median',
 'area-group_id-price_diff_median',
 'return_date_dayofyear-promotion_prog-price_mean',
 'group_id-order_date_dayofmonth-price_diff_median',
 'order_date_month',
 'product_name-begin_date_from2016-price_median',
 'unit-group_id-price_diff_mean',
 'source_1_price_diff_median',
 'group_id-order_date_from2016-price_diff_mean',
 'group_id-order_date_dayofmonth-price_median',
 'order_date_dayofyear-order_date_from2016-price_diff_median',
 'order_date_dayofyear-group_id-predays_median',
 'begin_date_month_predays_min',
 'order_date_dayofyear-begin_date_from2016-predays_median',
 'order_date_dayofyear-group_id-predays_diff_median',
 'area_price_mean',
 'order_date_dayofmonth-source_2-price_diff_mean',
 'area-product_name-price_median',
 'product_name-sub_line-price_mean',
 'order_date_dayofyear-begin_date_from2016-predays_diff_median',
 'return_date_month_predays_diff_median',
 'product_name-begin_date_from2016-price_diff_median',
 'group_id-order_date_from2016-price_mean',
 'group_id-begin_date_dayofyear-predays_diff_median',
 'begin_date_quarter_predays_diff_median',
 'begin_date_from2016',
 'begin_date_dayofweek_price_median',
 'return_date_dayofweek-group_id-predays_median',
 'product_name-source_1-price_median',
 'ticketsischeap_days',
 'order_date_dayofmonth-source_1-predays_diff_median',
 'return_date_dayofweek_predays_skew',
 'begin_date_dayofyear-order_date_from2016-predays_diff_mean',
 'product_name-source_2-price_median',
 'order_date_dayofyear-begin_date_dayofyear-predays_diff_median',
 'product_name-group_id-predays_diff_mean',
 'order_date_dayofyear-group_id-price_diff_mean',
 'product_name-source_1-price_mean',
 'group_id-promotion_prog-predays_diff_mean',
 'people_amount-source_2-price_diff_median',
 'begin_date_dayofyear-begin_date_from2016-price_diff_mean',
 'product_name-return_date_dayofyear-price_diff_mean',
 'sub_line_price_median',
 'order_date_from2016-begin_date_from2016-predays_diff_median',
 'order_date_dayofyear-begin_date_from2016-predays_mean',
 'promotion_prog-source_1-price_median',
 'ticketsischeap_days_price_diff_median',
 'promotion_prog-begin_date_from2016-predays_mean',
 'order_date_dayofyear-group_id-price_median',
 'ticketsischeap_week',
 'group_id-begin_date_dayofyear-predays_mean',
 'order_date_dayofyear_price_diff_median',
 'group_id-begin_date_from2016-price_mean',
 'return_date_dayofweek-order_date_dayofweek-predays_diff_mean',
 'order_date_dayofyear-group_id-predays_mean',
 'return_date_dayofweek-order_date_dayofweek-predays_diff_median',
 'return_date_dayofyear-group_id-predays_diff_median',
 'group_id-people_amount-price_diff_median',
 'group_id-begin_date_dayofyear-price_diff_mean',
 'source_2_price_diff_mean',
 'group_id-begin_date_from2016-predays_diff_mean',
 'begin_date_dayofyear-order_date_from2016-predays_mean',
 'return_date_dayofweek_predays_diff_median',
 'days_predays_median',
 'return_date_dayofyear-group_id-price_median',
 'source_2_price_max',
 'product_name-group_id-predays_median',
 'return_date_dayofyear-group_id-price_diff_median',
 'order_date_dayofyear-group_id-price_mean',
 'area-sub_line-price_diff_median',
 'product_name-return_date_dayofyear-price_mean',
 'order_date_dayofyear-return_date_dayofyear-predays_median',
 'promotion_prog-begin_date_from2016-price_mean',
 'premonths',
 'sub_line-promotion_prog-price_median',
 'return_date_dayofweek-source_2-predays_diff_mean',
 'group_id_price_median',
 'group_id-order_date_from2016-predays_mean',
 'return_date_dayofweek-group_id-predays_mean',
 'ticketsisexpen_week_price_diff_median',
 'return_date_dayofyear-promotion_prog-price_median',
 'group_id-begin_date_dayofyear-price_diff_median',
 'group_id-order_date_dayofmonth-price_diff_mean',
#  'predays',
 'product_name-begin_date_from2016-price_diff_mean',
 'ticketsischeap_week_predays_mean',
 'group_id-source_2-price_diff_median',
 'begin_date_dayofweek_predays_diff_mean',
 'order_date_month_price_diff_median',
 'people_amount_price_diff_median',
 'begin_date_quarter_predays_median',
 'sub_line-group_id-price_median',
 'ticketsischeap_week_predays_diff_mean']

In [ ]:
order.drop(columns=del_column,inplace=True)

In [ ]:
price_column = ['people_amount', 'unit', 'source_1', 'source_2']

predays_column = ['order_date_dayofweek', 'unit', 'source_1', 'people_amount', 'source_2', 'group_id',
                  'order_date_from2016', 'order_date_dayofyear', 'return_date_dayofweek']

In [ ]:
ts = time.time()
from itertools import combinations
methods = ['median','mean']
for column in list(combinations(price_column,3)):
    print(column)
    for method in methods: 
        order[column[0]+'='+column[1]+'='+column[2]+'=price_'+method] = order.groupby([column[0],column[1],column[2]])['price'].transform(method).astype(float)
        order[column[0]+'='+column[1]+'='+column[2]+'=price_diff_'+method] = order['price'] - order[column[0]+'='+column[1]+'='+column[2]+'=price_'+method]
for column in list(combinations(predays_column,3)):
    print(column)
    for method in methods:  
        order[column[0]+'='+column[1]+'='+column[2]+'=predays_'+method] = order.groupby([column[0],column[1],column[2]])['predays'].transform(method).astype(float)
        order[column[0]+'='+column[1]+'='+column[2]+'=predays_diff_'+method] = order['predays'] - order[column[0]+'='+column[1]+'='+column[2]+'=predays_'+method]
time.time()-ts

In [ ]:
order = pd.read_pickle('order1342_new.pkl')#round 3 -> 4

In [ ]:
del_column = ['group_id=order_date_from2016=return_date_dayofweek=predays_diff_median',
 'source_2=group_id=order_date_dayofyear=predays_mean',
 'source_2=group_id=order_date_dayofyear=predays_diff_mean',
 'order_date_from2016=order_date_dayofyear=return_date_dayofweek=predays_diff_median',
 'unit=group_id=order_date_dayofyear=predays_diff_mean',
 'group_id=order_date_dayofyear=return_date_dayofweek=predays_diff_mean',
 'group_id=order_date_dayofyear=return_date_dayofweek=predays_mean',
 'unit=group_id=order_date_dayofyear=predays_mean',
 'group_id=order_date_dayofyear=return_date_dayofweek=predays_diff_median',
 'source_2=order_date_from2016=order_date_dayofyear=predays_diff_median',
 'group_id=order_date_dayofyear=return_date_dayofweek=predays_median',
 'unit=group_id=order_date_dayofyear=predays_diff_median',
 'group_id=order_date_from2016=return_date_dayofweek=predays_diff_mean',
 'group_id=order_date_from2016=return_date_dayofweek=predays_median',
 'unit=group_id=order_date_dayofyear=predays_median',
 'unit=group_id=order_date_from2016=predays_diff_mean',
 'unit=group_id=order_date_from2016=predays_mean',
 'unit=group_id=order_date_from2016=predays_diff_median',
 'unit=group_id=order_date_from2016=predays_median',
 'source_2=group_id=order_date_dayofyear=predays_median',
 'group_id=order_date_from2016=order_date_dayofyear=predays_median',
 'group_id=order_date_from2016=order_date_dayofyear=predays_diff_median',
 'group_id=order_date_from2016=order_date_dayofyear=predays_mean',
 'group_id=order_date_from2016=order_date_dayofyear=predays_diff_mean',
 'group_id=order_date_from2016=return_date_dayofweek=predays_mean',
 'source_2=group_id=order_date_dayofyear=predays_diff_median',
 'unit=order_date_from2016=order_date_dayofyear=predays_diff_mean',
 'source_2=group_id=order_date_from2016=predays_diff_mean',
 'source_1=group_id=order_date_from2016=predays_median',
 'people_amount=group_id=order_date_from2016=predays_mean',
 'people_amount=group_id=order_date_from2016=predays_diff_median',
 'people_amount=group_id=order_date_from2016=predays_median',
 'order_date_dayofweek=source_2=order_date_dayofyear=predays_median',
 'order_date_dayofweek=source_1=order_date_dayofyear=predays_diff_median',
 'source_1=group_id=order_date_from2016=predays_diff_median',
 'people_amount=group_id=order_date_dayofyear=predays_median',
 'source_1=group_id=order_date_from2016=predays_mean',
 'source_1=group_id=order_date_from2016=predays_diff_mean',
 'source_1=group_id=order_date_dayofyear=predays_median',
 'source_1=group_id=order_date_dayofyear=predays_diff_median',
 'source_1=group_id=order_date_dayofyear=predays_mean',
 'source_1=order_date_from2016=order_date_dayofyear=predays_diff_median',
 'people_amount=group_id=order_date_from2016=predays_diff_mean',
 'people_amount=group_id=order_date_dayofyear=predays_diff_median',
 'source_2=group_id=order_date_from2016=predays_mean',
 'order_date_dayofweek=group_id=order_date_dayofyear=predays_diff_median',
 'source_2=group_id=order_date_from2016=predays_diff_median',
 'source_2=group_id=order_date_from2016=predays_median',
 'source_1=order_date_from2016=order_date_dayofyear=predays_diff_mean',
 'order_date_dayofweek=group_id=order_date_dayofyear=predays_diff_mean',
 'order_date_dayofweek=group_id=order_date_dayofyear=predays_mean',
 'people_amount=order_date_from2016=order_date_dayofyear=predays_diff_mean',
 'people_amount=group_id=order_date_dayofyear=predays_mean',
 'order_date_dayofweek=group_id=order_date_dayofyear=predays_median',
 'order_date_dayofweek=group_id=order_date_from2016=predays_diff_mean',
 'order_date_dayofweek=group_id=order_date_from2016=predays_mean',
 'order_date_dayofweek=group_id=order_date_from2016=predays_diff_median',
 'order_date_dayofweek=group_id=order_date_from2016=predays_median',
 'people_amount=group_id=order_date_dayofyear=predays_diff_mean',
 'source_1=group_id=order_date_dayofyear=predays_diff_mean',
 'people_amount=order_date_from2016=order_date_dayofyear=predays_diff_median']

In [ ]:
order.drop(columns=del_column,inplace=True)

In [ ]:
predays_column = ['order_date_dayofweek', 'unit', 'source_1', 'people_amount',
                  'source_2', 'group_id', 'return_date_dayofweek']

In [ ]:
ts = time.time()
from itertools import combinations
methods = ['median','mean']
for column in list(combinations(predays_column,4)):
    print(column)
    for method in methods:  
        order[column[0]+'~'+column[1]+'~'+column[2]+'~'+column[3]+'~predays_'+method] = order.groupby([column[0],column[1],column[2],column[3]])['predays'].transform(method).astype(float)
        order[column[0]+'~'+column[1]+'~'+column[2]+'~'+column[3]+'~predays_diff_'+method] = order['predays'] - order[column[0]+'~'+column[1]+'~'+column[2]+'~'+column[3]+'~predays_'+method]
time.time()-ts

In [ ]:
order = pd.read_pickle('order1421_newfillna.pkl')#round 3 -> 4

In [ ]:
ts = time.time()
train_1=train.merge(order,on='order_id')
test_1=test.merge(order,on='order_id')

# First predict by lightgbm and by cv
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve
import numpy as np
folds = StratifiedKFold(n_splits= 5, shuffle=True, random_state=666)
oof_preds = np.zeros((train_1.shape[0],4))
sub_preds = np.zeros((test_1.shape[0],4))
feature_importance_df = pd.DataFrame()
feats = [f for f in train_1.columns if f not in ['order_id','deal_or_not','order_date','begin_date','return_date']]
print ('feats:' + str(len(feats)),feats)
time.time()-ts

In [ ]:
ts = time.time()
# feats = target_column
oof_preds = np.zeros(train_1.shape[0])
sub_preds = np.zeros(test_1.shape[0])
for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_1[feats], train_1['deal_or_not'])):
    train_x, train_y = train_1[feats].iloc[train_idx], train_1['deal_or_not'].iloc[train_idx]
    valid_x, valid_y = train_1[feats].iloc[valid_idx], train_1['deal_or_not'].iloc[valid_idx]
    print("Train Index:",train_idx,",Val Index:",valid_idx)

    params = {
        'nthread': 32,
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'learning_rate': 0.1,
        'num_leaves': 100,
        'max_depth': 8,
    #     'bagging_fraction': 0.9,
        'feature_fraction': 0.7,
    #     'bagging_freq': 1,
        'min_split_gain': 0.1,
        'min_data_in_leaf': 20,
        'min_child_weight': 0.01,
        'reg_alpha': 10,
        'reg_lambda': 10
    }

    if n_fold >= 0:
        dtrain = lgb.Dataset(train_x, label=train_y)
        dval = lgb.Dataset(valid_x, label=valid_y, reference=dtrain)
        
        bst = lgb.train(
        params, dtrain, num_boost_round=5000,
        valid_sets=[dval], early_stopping_rounds=100, verbose_eval=100)
        
       
        oof_preds[valid_idx] = bst.predict(valid_x, num_iteration=bst.best_iteration)
        
        sub_preds += bst.predict(test_1[feats], num_iteration=bst.best_iteration) / folds.n_splits
        # Make the feature importance dataframe
        gain = bst.feature_importance('gain')
        fold_importance_df = pd.DataFrame({'feature':bst.feature_name(),
        'split':bst.feature_importance('split'),
        'gain':100*gain/gain.sum(),
        'fold':n_fold,
        }).sort_values('gain',ascending=False)
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))
        gc.collect()
time.time()-ts

In [ ]:
print(roc_auc_score(train_1['deal_or_not'], oof_preds))

In [ ]:
test.iloc[:,1]=sub_preds
test.to_csv('submission1421_dartparams.csv', index=False)

In [ ]:
# 0.751926 0.755864 0.751104 0.753489 0.755820
# 0.75315141731617
# params = {
#     'nthread': 32,
#     'boosting_type': 'dart',
#     'objective': 'binary',
#     'metric': 'auc',
#     'learning_rate': 0.1,
#     'num_leaves': 100,
#     'max_depth': 8,
#     'feature_fraction': 0.7,
#     'min_split_gain': 0.1,
#     'min_data_in_leaf': 100,
#     'min_child_weight': 0.01,
#     'reg_alpha': 10,
#     'reg_lambda': 10
#     'max_drop':100
#     }

In [ ]:
# 0.7529802796163858
# 0.750538 0.755985 0.751708 0.752874 0.754835
# params = {
#     'nthread': 32,
#     'boosting_type': 'dart',
#     'objective': 'binary',
#     'metric': 'auc',
#     'learning_rate': 0.1,
#     'num_leaves': 100,
#     'max_depth': 8,
# #     'bagging_fraction': 0.9,
#     'feature_fraction': 0.7,
# #     'bagging_freq': 1,
#     'min_split_gain': 0.1,
#     'min_data_in_leaf': 20,
#     'min_child_weight': 0.01,
#     'reg_alpha': 10,
#     'reg_lambda': 10}

In [ ]:
# 0.748891 0.752511 0.748307 0.749895 0.751135
# 0.7500506275424007

# params = {
#     'nthread': 32,
#     'boosting_type': 'gbdt',
#     'objective': 'binary',
#     'metric': 'auc',
#     'learning_rate': 0.1,
#     'num_leaves': 100,
#     'max_depth': 8,
# #     'bagging_fraction': 0.9,
#     'feature_fraction': 0.7,
# #     'bagging_freq': 1,
#     'min_split_gain': 0.1,
#     'min_data_in_leaf': 20,
#     'min_child_weight': 0.01,
#     'reg_alpha': 10,
#     'reg_lambda': 10
# }

In [ ]:
# test #1
# 0.7491773232853496

# params = {
#     'nthread': 32,
#     'boosting_type': 'gbdt',
#     'objective': 'binary',
#     'metric': 'auc',
#     'learning_rate': 0.1,
#     'num_leaves': 700,
#     'max_depth': 10,
#     'bagging_fraction': 0.8,
#     'feature_fraction': 0.7,
# #     'bagging_freq': 1,
#     'min_split_gain': 0.1,
#     'min_data_in_leaf': 20,
#     'min_child_weight': 0.001,
#     'reg_alpha': 20,
#     'reg_lambda': 20
# }

In [ ]:
# 0.746962 0.751682 0.748047 0.748269 0.750217
# print(roc_auc_score(train_1['deal_or_not'], oof_preds))
# 0.7488902085263649
# round 4 [1421] +fillna

In [ ]:
# 0.745124 0.748945 0.746636 0.747339 0.748917
# print(roc_auc_score(train_1['deal_or_not'], oof_preds))
# 0.7472887557949323
# round 4 [1421]

In [ ]:
# 0.745037 0.750224 0.745587 0.747987 0.749872
# print(roc_auc_score(train_1['deal_or_not'], oof_preds))
# 0.747713436954943
# round 3 [1342] + fillna

In [ ]:
# 0.746426 0.748549 0.746612 0.746560 0.749835
# print(roc_auc_score(train_1['deal_or_not'], oof_preds))
# 0.7475275449955711
# round 3 [1342]

In [ ]:
# 0.742813 0.746924 0.743132 0.743785 0.746671
# print(roc_auc_score(train_1['deal_or_not'], oof_preds))
# 0.7445765427478643
# round 2 [1340]

In [ ]:
# 0.731972 0.735486 0.732104 0.732330 0.734632
# print(roc_auc_score(train_1['deal_or_not'], oof_preds))
# 0.7332703314914079
# round 1 [493]

In [ ]:
feature_importance_df

In [ ]:
del_column = []
for i in range(4):
    del_column = del_column + list(feature_importance_df[(feature_importance_df.fold==i)&(feature_importance_df.gain==0)].feature)
# del_column = list(set(del_column))
len(del_column)

In [ ]:
count_dict = dict()
for item in del_column:
    if item in count_dict:
        count_dict[item] += 1
    else:
        count_dict[item] = 1
        
del_column = []
for i in count_dict:
    if count_dict[i]==(4) or count_dict[i]==(3) :
#         print(i, count_dict[i])
        del_column.append(i)
len(del_column)

In [ ]:
target_column = []
for i in range(4):
    target_column = target_column + list(feature_importance_df[(feature_importance_df.fold==i)&(feature_importance_df.gain!=0)][:10].feature)
target_column = list(set(target_column))
len(target_column)

In [ ]:
predays_column = []
price_column = []
for i in target_column:
    if '~' in i:
        print(i)
        if 'predays' in i:
            predays_column.append(i.split('~')[0])
            predays_column.append(i.split('~')[1])
            predays_column.append(i.split('~')[2])
predays_column = list(set(predays_column)); price_column = list(set(price_column))
print(len(predays_column),len(price_column))

In [ ]:
# for 33 prepare
predays_column = []
price_column = []
for i in target_column:
    if '=' in i:
#         print(i.split('='))
        if 'price' in i:
            price_column.append(i.split('=')[0])
            price_column.append(i.split('=')[1])
            price_column.append(i.split('=')[2])
        if 'predays' in i:
            predays_column.append(i.split('=')[0])
            predays_column.append(i.split('=')[1])
            predays_column.append(i.split('=')[2])
predays_column = list(set(predays_column)); price_column = list(set(price_column))
print(len(predays_column),len(price_column))

In [ ]:
target_column = []
for i in range(4):
    target_column = target_column + list(feature_importance_df[(feature_importance_df.fold==i)&(feature_importance_df.gain!=0)][:38].feature)
target_column = list(set(target_column))
len(target_column)

In [ ]:
# for 33 prepare
predays_column = []
price_column = []
for i in target_column:
    if '-' in i:
        if 'price' in i:
            price_column.append(i.split('-')[0])
            price_column.append(i.split('-')[1])
        if 'predays' in i:
            predays_column.append(i.split('-')[0])
            predays_column.append(i.split('-')[1])
predays_column = list(set(predays_column)); price_column = list(set(price_column))
print(len(predays_column),len(price_column))

In [ ]:
del_column = []
for i in range(4):
    del_column = del_column + list(feature_importance_df[(feature_importance_df.fold==i)][1070:].feature)
del_column = list(set(del_column))
len(del_column)

In [ ]:
predays_column = []
price_column = []
for i in target_column:
    if 'price' in i:
        if 'diff' in i.split('_')[:]:
                price_column.append('_'.join(i.split('_')[:-3]))
        else:
                price_column.append('_'.join(i.split('_')[:-2]))
for i in target_column:
    if 'predays' in i:
        if 'diff' in i.split('_')[:]:
                predays_column.append('_'.join(i.split('_')[:-3]))
        else:
                predays_column.append('_'.join(i.split('_')[:-2]))
for i in target_column:
    if 'predays' not in i:
        if 'price' not in i:
            predays_column.append(i);price_column.append(i)
predays_column = list(set(predays_column)); price_column = list(set(price_column))
print(len(predays_column),len(price_column))

In [ ]:
del_column = []
for i in range(4):
    del_column = del_column + list(feature_importance_df[(feature_importance_df.fold==i)&(feature_importance_df.gain==0)].feature)
len(del_column)

In [ ]:
count_dict = dict()
for item in del_column:
    if item in count_dict:
        count_dict[item] += 1
    else:
        count_dict[item] = 1
        
del_column = []
for i in count_dict:
    if count_dict[i]==(4 or 5):
#         print(i, count_dict[i])
        del_column.append(i)
len(del_column)